# Kohya ss py3.11


# 启动Webui

注意，如果你使用多卡训练，请先配置accelerate（会用多卡的相信你会配置了）
```bash
accelerate config
```
本镜像在RTX4090单卡上配置，如果训练中报错，也可以尝试配置acclerate


# 运行下面的代码块从云端更新`start.ipynb`（本文档）
### 更新完后请重新刷新jupyterlab的文件管理然后打开此文档 `/root/start.ipynb`
### 现在运行的版本是 24.02.19   stable-cascade 特别版本 已最新

In [ ]:
!rm -rf /root/start.ipynb && wget https://drive.t4wefan.pub/d/windsing/wingsing/start.ipynb

# 可选：下载训练所需模型（SDXL/SD1.5）
(SDXL官模默认已下载到/root/ckpts/sd_xl_base_1.0.safetensors，这里不用运行了)

下面下载的模型建议保存到/root/autodl-tmp/ckpts路径

## stable-cascade版本删除了sd_xl_base_1.0.safetensors，需要手动下载

In [ ]:
!source /etc/network_turbo && wget https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors -P /root/ckpts

## 最新：下载stable-cascade训练底模（stable-cascade版本内置）

In [ ]:
!source /etc/network_turbo && wget https://huggingface.co/stabilityai/stable-cascade/resolve/main/stage_c_bf16.safetensors -P /root/autodl-tmp/ckpts

## 1.下载SDXL动漫推荐底模 animagine-xl-3.0

In [ ]:
!source /etc/network_turbo && wget https://huggingface.co/cagliostrolab/animagine-xl-3.0/resolve/main/animagine-xl-3.0.safetensors -P /root/ckpts

## 2.下载SD1.5动漫推荐底模 animefull-latest-ckpt （nai）

In [ ]:
!source /etc/network_turbo && https://huggingface.co/deepghs/animefull-latest-ckpt/resolve/main/model.ckpt  -P /root//ckpts

-----------------------------------------------------------------------------------------

# 用公开服务端口启动kohya_ss，运行下面这一行  <span style="color:red; "> （新手推荐）  </span>


2024.01.04更新： 使用https://hf-mirror.com/ 网站下载抱脸依赖，训练不会再报错，感谢镜像站！功德无量

本镜像已经内置了catcat-xl训练集(/root/dataset/catcat)，可以用于测试是否工作正常

启动成功后，返回控制台，点击你的实例的“自定义服务”进入UI即可



训练遇到问题，翻到下面的<span style="color:red; font-size:larger;">SDXL的训练报错”栏目</span>  
 
当前版本已支持SDXL大模型训练，支持中文界面


In [ ]:
# 默认模型已下载，在/root/ckpts/sd_xl_base_1.0_0.9vae.safetensors
!mkdir /root/autodl-tmp/current #创建临时文件夹存储训练中数据
!cd /root/kohya_ss && HF_ENDPOINT=https://hf-mirror.com ./gui.sh --language zh-CN --server_port 6006 --listen 0.0.0.0 --headless 

# 点击下面关闭学术加速（如果有需要）
（Stable-diffusion-webui启动时默认学术加速，为了方便从抱脸下载依赖，）

In [ ]:
!unset http_proxy && unset https_proxy

# 启动 Stable-diffusion-webui 来调试模型
(适合新手，默认安装tagger可以用来给训练集打标，默认6006端口，也可以改成自己的)

## 1.移动sd到数据盘 ，系统盘空间不够只能放一个大模型

In [ ]:
!mv /root/stable-diffusion-webui /root/autodl-tmp/stable-diffusion-webui

## 2.启动 stable-diffusion-webui

In [ ]:
!source /etc/network_turbo && cd  /root/autodl-tmp/stable-diffusion-webui && python launch.py --port 6006 --xformers --enable-insecure-extension-access --listen --no-half

# 启动 ComfyUI 来调试模型
(适合专业用户，默认6006端口，也可以改成自己的)

## 直接在系统盘启动comfyui（不推荐）

In [ ]:
!source /etc/network_turbo && cd  /root/ComfyUI  && python main.py --preview-method auto --port 6006 --listen 0.0.0.0

## 1.移动comfyui到数据盘，拥有更多空间

In [ ]:
!mv /root/ComfyUI /root/autodl-tmp/ComfyUI

## 2.启动ComfyUI

In [ ]:
!source /etc/network_turbo && cd  /root/autodl-tmp/ComfyUI  && python main.py --preview-method auto --port 6006 --listen 0.0.0.0



## 手动部署（本镜像自带环境，理论上可以直接运行，推荐py3.11）

In [ ]:
# 如果你同时使用多个gpu
!export MKL_SERVICE_FORCE_INTEL=1
!export MKL_THREADING_LAYER=GNU

克隆仓库

In [ ]:
# 启动学术加速
!source /etc/network_turbo

In [ ]:
!git clone -b sdxl-dev https://github.com/bmaltais/kohya_ss.git

安装pytorch（根据当前镜像最高版本来选择）

In [ ]:
#cu117
#!conda install pytorch torchvision torchaudio pytorch-cuda=11.7 -c pytorch -c nvidia

#cu118
!conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia

安装需要的依赖

In [ ]:
!cd /root/konya_ss
!pip install -r requirements.txt
!pip install xformers==0.0.20 bitsandbytes==0.35.0 tensorboard==2.12.3 tensorflow==2.12.0

如果你使用上一个代码块时依赖不全，尝试这个

In [ ]:
!pip install accelerate==0.19.0 albumentations==1.3.0 altair==4.2.2 dadaptation==3.1 diffusers[torch]==0.18.2 easygui==0.98.3 einops==0.6.0 fairscale==0.4.13 ftfy==6.1.1 gradio==3.36.1 huggingface-hub==0.15.1 lion-pytorch==0.0.6 lycoris_lora==1.8.0 invisible-watermark==0.2.0 open-clip-torch==2.20.0 opencv-python==4.7.0.68 prodigyopt==1.0 pytorch-lightning==1.9.0 rich==13.4.1 safetensors==0.3.1 timm==0.6.12 tk==0.1.0 toml==0.10.2 transformers==4.30.2 voluptuous==0.13.1 wandb==0.15.0 xformers==0.0.20 bitsandbytes==0.35.0 tensorboard==2.12.3 tensorflow==2.12.0

In [ ]:
# 记得取消学术加速
!unset http_proxy && unset https_proxy



# 一些附赠软件
### frp 方便用于内网穿透，需要手写配置文件
### gohttpserver 轻量级文件管理web，可以方便的打包和多线程下载文件
### Eugeoter/sd-dataset-manager 最舒服的可视化数据集处理工具

Eugeoter/sd-dataset-manager支持：自定义过滤、打标、删标、批量去角色特征词、美学评分等功能

## 1.超级好用！可以通过下面的gohttpserver 来上传训练集和打包下载模型文件！

In [ ]:
# 启动 gohttpserver 会占用6006端口，可以自己改端口。

!cd /root && ./gohttpserver --port 6006 --upload

## 2.启动Eugeoter/sd-dataset-manager 训练集处理工具
github链接：https://github.com/Eugeoter/sd-dataset-manager

有帮助的话，记得给作者点个star~

1.**更新训练集处理工具** 

In [ ]:
!cd /root/sd-dataset-manage  &&  git pull

2.**启动训练集处理工具**  
“path/to/folder” 替换成你想处理的训练集路径,支持同时处理多个文件夹

In [ ]:
!source /etc/network_turbo && cd /root/sd-dataset-manager && python api.py --source 'path/to/folder' --write_to_txt --language cn --port 6006

##  文件夹映射（共用模型文件） ln -s A文件夹的路径 B文件夹的路径

In [ ]:
!ln -s /root/ckpts /root/stable-diffusion-webui/models/Stable-diffusion
!ln -s /root/ckpts /root/ComfyUI/models/checkpoints

# SDXL的训练报错栏目


<span style="color:red; font-size:larger;">#1.怀疑自己参数有问题，看这个（针对lora训练）</span>

可以试试加载“nobug.json”文件，路径是 `/root/example/SDXL_lora.json`

kohya_ss的脚本可以很方便地加载别人的训练参数，没记错的话点开UI左上角的的Configuration file 进去加载


记得改路径和其他参数！这个训练参数只是作为我个人的示范

部分优化器会导致SDXL训练报错,目前默认的优化器和神童优化器是安全的（待补充）

b站视频来的提醒一下，如果上面使用了带bf16的预设，下面就不要勾选fp16相关选项，比如full fp16 training

<span style="color:red; font-size:larger;"># 2 .抱脸安装失败的,通过这行命令启动训练</span>

如果报错出现什么6006端口已经被占用，可能是你之前的训练训练还开着。

先按左上角“内核”-“重启内核并清除所有输出” 关掉之前的端口,

然后把这个命令复制粘贴到**终端**执行

`cd /root/kohya_ss && HF_ENDPOINT=https://hf-mirror.com ./gui.sh --language zh-CN  --server_port 6006 --listen 0.0.0.0 --headless `

终端在哪里？
启动页——其他——终端

<span style="color:red; font-size:larger;">#3.报错：NameError: name 'CUDA_ RUNTIME_ LIB’ is not defined</span>

出现这个报错，建议更换训练的显卡，CUDA建议12.0，其他版本都容易奇奇怪怪的报错


<span style="color:red; font-size:larger;"># 4 .关于SDXL的大模型训练</span>

1.SDXL大模型训练需要带上vae训练

--不带vae当前版本会发生detected in latents 的报错，vae我放在`/root/vae/sdxl_vae.safetensors`

2.SDXL大模型训练分UI训练和脚本训练

-- UI训练SDXL大模型的流程跟lora差不多，在kohya_ss_gui右上角切换到“Dreambooth”就行，有一个参考的配置文件`/root/example/SDXL_db.json`（能动，能训练，但是显存占用高），可以在这个配置上自行修改其他参数
   
3.SDXL大模型脚本训练

--   脚本训练包括青龙脚本和其他的一些魔改脚本，脚本需要自己上传，个人感觉，脚本训练比UI训练更省显存，我现在都是用脚本训练的， 镜像环境是支持大模型训练的，有动手能力的修改一下脚本里面相关的文件路径，大概就能直接用了（稍微有点麻烦）,"train_sdxl.sh"可以换成你的脚本实际的文件名字

4.SDXL大模型训练的显存和显卡

--大模型训练很吃显存，大概是24G以上吧，目前我租的显卡是A40(48G)和A100（40G和80G）


# ai交流与问题反馈
镜像交流群：906520028 ； SDXL模型训练交流群：914818692





# 运行下面的代码启动脚本训练 （example）
根据需要修改train_sdxldb 文件内的参数

大模型路径 pretrained_model

保存模型名称   output_name

数据集路径   train_data_dir

保存模型的路径  output_dir

日志的路径 logging_dir

其他的自己看需要修改

**启动SDXL 大模型训练** 16G显存占用

In [ ]:
!cd /root && HF_ENDPOINT=https://hf-mirror.com  bash train_sdxldb.sh

**启动SDXL LORA训练**

In [ ]:
!cd /root &&  HF_ENDPOINT=https://hf-mirror.com bash train_sdxllora.sh

**启动自动批量训练**

In [ ]:
!cd /root/kohya_ss & chmod +x train.sh && HF_ENDPOINT=https://hf-mirror.com ./train.sh 